In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import mjaigym
from mjaigym.mjson import Mjson
from ml.custom_observer import SampleCustomObserver
from mjaigym.board import ArchiveBoard
from mjaigym.reward import KyokuScoreReward



# Load mjson

In [ ]:
mjson = Mjson.load('/data/mjson/test/201712/2017123123gm-00e1-0000-f75ac2b9.mjson')

# featurize mjson

In [ ]:
env = SampleCustomObserver(board=ArchiveBoard(), reward_calclator_cls=KyokuScoreReward)
env.reset()

states = []
for kyoku_id, kyoku in enumerate(mjson.game.kyokus):
    for index, action in enumerate(kyoku.kyoku_mjsons):
        next_state, reward, done, info = env.step(action)

        for player_id in range(4):
            next_state[player_id].calclate()
            if next_state[player_id].dahai_observation is not None:
                states.append((kyoku_id, index, player_id, next_state[player_id].dahai_observation))


# Get MLP middle filter output 

In [ ]:
from ml.model import Head34Value1SlModel

model = Head34Value1SlModel(
    in_channels=env.get_tsumo_observe_channels_num(),
    mid_channels=256,
    blocks_num=50,
    learning_rate=0.0001,
    batch_size=256
)

model.load("./cache/dahai.pth")

In [ ]:
dataset = []

for (kyoku_id, line_index, player_id, dahai_state) in states:
    p, v, v_mid = model.predict([dahai_state])
    dataset.append({
        "kyoku_id":kyoku_id,
        "line_index":line_index, 
        "player_id":player_id, 
        "p":np.argmax(p), 
        "v":v[0][0],
        "v_mid":v_mid[0]
        })

dataset = pd.DataFrame(dataset)

dataset

In [ ]:
%matplotlib inline
import umap
import time
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.manifold import TSNE

In [ ]:
start_time = time.time()
#embedding = umap.UMAP().fit_transform(np.vstack(dataset.v_mid))
tsne_model = TSNE(n_components=2)
embedding = tsne_model.fit_transform(np.vstack(dataset.v_mid))
interval = time.time() - start_time


fig = plt.figure(figsize=(16,6))
mark_list = ["o", "+", "s", ">"]
color_list = ["Red", "Blue", "Green", "Black"]
#  for color_id, color in enumerate(colors_list):
cmap = plt.get_cmap("Reds")

for i in range(len(dataset)):
    target_index = i
    kyoku_id = dataset.kyoku_id[target_index]
    player_id = dataset.player_id[target_index]
    mark = mark_list[player_id]
    mappable = plt.scatter(embedding[target_index,0], embedding[target_index,1], 
                           color=cmap(((kyoku_id-1)*4+player_id)/16), 
                           s=200, 
                           marker="$"+f"{str(i)}"+"$", 
                           vmin=0, vmax=1)

#fig.colorbar(mappable)
plt.legend([0,1,2,3])
plt.show()


In [ ]:
sns.lineplot(x='line_index', y='v', hue='player_id', data=dataset)